In [19]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
%matplotlib inline

In [20]:
boston_data=np.loadtxt("boston.csv",delimiter=",")

In [ ]:
#LDA

In [21]:
boston_data_50=np.copy(boston_data)
boston_data_50[:,-1]=(boston_data_50[:,-1]>=np.percentile(boston_data_50[:,-1],50)).astype(int)

In [23]:
def LDA_means(X,target):
        classes=list(np.unique(target))
        mu=np.zeros(shape=(len(classes),X.shape[1]))
        for x in classes:
            mu[x]=(np.apply_along_axis(np.mean,0,X[target==x]).reshape(1,X.shape[1]))
        return mu

In [24]:
def within_class_variance(X,mus,target):
    classes=list(np.unique(target))
    Sw=np.zeros((mus.shape[1],mus.shape[1]))
    i=1
    for x in classes:
        Swi=np.zeros((mus.shape[1],mus.shape[1]))
        for y in range(0,len(X[target==x])):
            Swi=np.add(Swi,np.subtract(X[target==x][y].reshape(13,1),mus[x].reshape(13,1)).dot(
                                  np.subtract(X[target==x][y].reshape(1,13),mus[x].reshape(1,13))))
        Sw=np.add(Sw,Swi)
    return Sw

In [25]:
def between_class_variance(X,mus,target):
    classes=list(np.unique(target))
    Sb=np.zeros((mus.shape[1],mus.shape[1]))
    mu=np.mean(X,0)
    for x in classes:
        Sb=np.add(Sb,np.multiply(np.subtract(mus[x],mu).reshape(13,1).dot(np.subtract(mus[x].reshape(1,13),mu.reshape(1,13))),
                                 len(X[target==x])))
    return Sb

In [26]:
def x_hat_func(Sw,Sb,Xtrain,Xtest):
    final_matrix  = np.dot(np.linalg.inv(Sw), Sb)
    W,V=np.linalg.eig(final_matrix)
    xhat=Xtrain.dot(V[:,np.argsort(abs(W))[-1]].real.reshape(Xtrain.shape[1],1))
    xhat_test=Xtest.dot(V[:,np.argsort(abs(W))[-1]].real.reshape(Xtest.shape[1],1))
    return abs(xhat),abs(xhat_test)

In [27]:
def accuracy(ytest,yhat):
    right=float((yhat==ytest).sum())
    accuracy=right/float(ytest.shape[0])
    return accuracy

In [28]:
def cross_validation(X):
    np.random.shuffle(X)
    index=[]
    for i in range(0,10):
        index.append(range(X.shape[0]*i/10,X.shape[0]*(i+1)/10))
    accura=[]
    accura_train=[]
    for i in range(0,len(index)):
        Xtest=np.copy(X[index[i],:-1])
        ytest=np.copy(X[index[i],-1])
        Xtrain=np.delete(X[:,:-1],index[i],axis=0)
        ytrain=np.delete(X[:,-1],index[i],axis=0)
        mus=LDA_means(Xtrain,ytrain)
        Sw=within_class_variance(Xtrain,mus,ytrain)
        Sb=between_class_variance(Xtrain,mus,ytrain)
        xhat,xhat_test=x_hat_func(Sw,Sb,Xtrain,Xtest)
        mean=[]
        for y in np.unique(ytrain):
            mean.append(np.mean(xhat[ytrain==y]))
        yhat=(xhat_test<=np.mean(mean)).astype(int)
        yhat_train=(xhat<=np.mean(mean)).astype(int)
        accura.append(accuracy(ytest.reshape(ytest.shape[0],1),yhat))
        accura_train.append(accuracy(ytrain.reshape(ytrain.shape[0],1),yhat_train))
    return accura,accura_train

In [30]:
acc,acc_train=cross_validation(boston_data_50)
error=list(1.0-np.array(acc))
error_train=list(1.0-np.array(acc_train))
print "Traing Error"
print error_train
print "------------"
print "Standard Deveation Train"
print np.std(error_train)
print "------------"
print "Train error mean"
print np.mean(error_train)
print "------------"
print "Test Error"
print error
print "------------"
print "Test error mean"
print np.mean(error)
print "------------"
print "Standard Deveation test"
print np.std(error)

Traing Error
[0.14254385964912286, 0.15164835164835166, 0.13157894736842102, 0.14505494505494509, 0.14065934065934071, 0.13596491228070173, 0.1428571428571429, 0.14473684210526316, 0.1428571428571429, 0.13846153846153841]
------------
Standard Deveation Train
0.00519321474277
------------
Train error mean
0.141636302294
------------
Test Error
[0.12, 0.058823529411764719, 0.26000000000000001, 0.13725490196078427, 0.21568627450980393, 0.18000000000000005, 0.11764705882352944, 0.19999999999999996, 0.13725490196078427, 0.11764705882352944]
------------
Test error mean
0.154431372549
------------
Standard Deveation test
0.0559014896714
